In [1]:
#This is a system which provides you with the current location of the user using its latitude and longitude
#as its location and plotting it on the map.

In [3]:
#importing necessary libraries
#system information
import requests
import json
req = requests.get('https://ipinfo.io/')
print(req.text)
#this is my current information.

{
  "ip": "157.37.147.92",
  "city": "Jaipur",
  "region": "Rajasthan",
  "country": "IN",
  "loc": "26.9196,75.7878",
  "org": "AS55836 Reliance Jio Infocomm Limited",
  "postal": "302001",
  "timezone": "Asia/Kolkata",
  "readme": "https://ipinfo.io/missingauth"
}


In [4]:
#converting the python dictionary above into a JSON string that can be written into a file.

data = requests.get('https://ipinfo.io/').json()
print(req.text)
print()
print()
print('This is my current location: ',data["loc"], data["city"])

{
  "ip": "157.37.147.92",
  "city": "Jaipur",
  "region": "Rajasthan",
  "country": "IN",
  "loc": "26.9196,75.7878",
  "org": "AS55836 Reliance Jio Infocomm Limited",
  "postal": "302001",
  "timezone": "Asia/Kolkata",
  "readme": "https://ipinfo.io/missingauth"
}


This is my current location:  26.9196,75.7878 Jaipur


In [5]:
# using map() to 
# perform conversion from strings to list of float values.
data_ = list(map(float,data['loc'].split(',')))
#data[0], data[1]

In [6]:
#for more detailed information we can execute this command

import requests
res = requests.get('https://api.ipdata.co?api-key=test').json()
res

{'ip': '157.37.147.92',
 'is_eu': False,
 'city': 'Jaipur',
 'region': 'Rajasthan',
 'region_code': 'RJ',
 'country_name': 'India',
 'country_code': 'IN',
 'continent_name': 'Asia',
 'continent_code': 'AS',
 'latitude': 26.9204,
 'longitude': 75.7882,
 'postal': '302012',
 'calling_code': '91',
 'flag': 'https://ipdata.co/flags/in.png',
 'emoji_flag': '🇮🇳',
 'emoji_unicode': 'U+1F1EE U+1F1F3',
 'asn': {'asn': 'AS55836',
  'name': 'Reliance Jio Infocomm Limited',
  'domain': 'jio.com',
  'route': '157.37.128.0/18',
  'type': 'isp'},
 'carrier': {'name': 'Reliance', 'mcc': '404', 'mnc': '09'},
 'languages': [{'name': 'Hindi', 'native': 'हिन्दी'},
  {'name': 'English', 'native': 'English'}],
 'currency': {'name': 'Indian Rupee',
  'code': 'INR',
  'symbol': 'Rs',
  'native': 'টকা',
  'plural': 'Indian rupees'},
 'time_zone': {'name': 'Asia/Kolkata',
  'abbr': 'IST',
  'offset': '+0530',
  'is_dst': False,
  'current_time': '2020-08-02T23:40:27.290391+05:30'},
 'threat': {'is_tor': False,


In [7]:
#plotting data on map with different kinds of plane using folium
import folium
from branca.element import Figure
mymap = folium.Map(location=[data_[0], data_[1]], zoom_start=6)
folium.Marker((data_[0], data_[1]), popup=data["city"]).add_to(mymap)
fig2=Figure(width=550,height=350)
fig2.add_child(mymap)
folium.TileLayer('Stamen Terrain').add_to(mymap)
folium.TileLayer('Stamen Toner').add_to(mymap)
folium.TileLayer('Stamen Water Color').add_to(mymap)
folium.TileLayer('cartodbpositron').add_to(mymap)
folium.TileLayer('cartodbdark_matter').add_to(mymap)
folium.LayerControl().add_to(mymap)

In [11]:
#mapping location
mymap

In [13]:
#Since we are not using any google api's, so we try to scrape weather data from the browser
from bs4 import BeautifulSoup as bs
import requests
#defining browsers
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
# UK english
LANGUAGE = "en-UK,en;q=0.5"

def get_weather_data(url):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT
    session.headers['Accept-Language'] = LANGUAGE
    session.headers['Content-Language'] = LANGUAGE
    html = session.get(url)
    # create a new soup
    soup = bs(html.text, "html.parser")
    # store all results on this dictionary
    result = {}
    # extract region
    result['region'] = soup.find("div", attrs={"id": "wob_loc"}).text
    # extract temperature now
    result['temp_now'] = soup.find("span", attrs={"id": "wob_tm"}).text
    # get the day and hour now
    result['dayhour'] = soup.find("div", attrs={"id": "wob_dts"}).text
    # get the actual weather
    result['weather_now'] = soup.find("span", attrs={"id": "wob_dc"}).text
    # get the precipitation
    result['precipitation'] = soup.find("span", attrs={"id": "wob_pp"}).text
    # get the % of humidity
    result['humidity'] = soup.find("span", attrs={"id": "wob_hm"}).text
    # extract the wind
    result['wind'] = soup.find("span", attrs={"id": "wob_ws"}).text
    # get next few days' weather
    next_days = []
    days = soup.find("div", attrs={"id": "wob_dp"})
    for day in days.findAll("div", attrs={"class": "wob_df"}):
        # extract the name of the day
        day_name = day.find("div", attrs={"class": "QrNVmd"}).attrs['aria-label']
        # get weather status for that day
        weather = day.find("img").attrs["alt"]
        temp = day.findAll("span", {"class": "wob_t"})
        # maximum temparature in Celsius, use temp[1].text if you want fahrenheit
        max_temp = temp[0].text
        # minimum temparature in Celsius, use temp[3].text if you want fahrenheit
        min_temp = temp[2].text
        next_days.append({"name": day_name, "weather": weather, "max_temp": max_temp, "min_temp": min_temp})
    # append to result
    result['next_days'] = next_days
    return result
    
loc_to_find = " https://www.google.com/search?q=jaipur+weather&oq="+data["city"]+"+weather&aqs=chrome..69i57.15052j0j1&sourceid=chrome&ie=UTF-8"+data["city"]
get_weather_data(loc_to_find)

{'region': 'Jaipur, Rajasthan',
 'temp_now': '30',
 'dayhour': 'Sunday, 11:00 pm',
 'weather_now': 'Haze',
 'precipitation': '3%',
 'humidity': '72%',
 'wind': '8 km/h',
 'next_days': [{'name': 'Sunday',
   'weather': 'Partly cloudy',
   'max_temp': '36',
   'min_temp': '27'},
  {'name': 'Monday',
   'weather': 'Partly cloudy',
   'max_temp': '36',
   'min_temp': '27'},
  {'name': 'Tuesday',
   'weather': 'Partly cloudy',
   'max_temp': '34',
   'min_temp': '27'},
  {'name': 'Wednesday',
   'weather': 'Partly cloudy',
   'max_temp': '35',
   'min_temp': '27'},
  {'name': 'Thursday',
   'weather': 'Partly cloudy',
   'max_temp': '33',
   'min_temp': '27'},
  {'name': 'Friday',
   'weather': 'Partly cloudy',
   'max_temp': '32',
   'min_temp': '26'},
  {'name': 'Saturday',
   'weather': 'Partly cloudy',
   'max_temp': '33',
   'min_temp': '27'},
  {'name': 'Sunday',
   'weather': 'Partly cloudy',
   'max_temp': '33',
   'min_temp': '27'}]}